# [Insert Creative Title HERE]

## Introduction and Motivation

## Required Libraries

Intro to Pandas:
https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html

Learning the Spotify API through spotipy:
https://spotipy.readthedocs.io/en/latest/#


In [ ]:
import sys
import math
import pandas as pd
import spotipy
import spotipy.util as util

## Data Collection and Tidying
//TODO Talk about general authentication flow. Link example resource. Explain how to get the spotify username
//TODO talk about how we chose what columns to keep. Talk about how data is relatively simple and no need to reorganize

In [27]:
print('What music would you like to include in the dataset?')
print()
print('Press 1 for saved/liked tracks')
print('Press 2 for a sample dataset')
print()
choice = int(input("Value: "))

while choice < 1 or choice > 2:
    print()
    print('Not a valid choice XD, try again')
    print()
    choice = int(input('Value: '))

if choice == 1:
    # Generate authorization token for reading user library
    scope = 'user-library-read'
    client_id = 'c3aacebd1b0d4a7ca415ca317ec72531'
    client_secret = '343a5709a346489d9aa013f32f00cfe4'
    redirect_uri = 'http://google.com/'

    print("Please Enter Username")
    username = str(input("Username: ")) #12124976904

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)

    else:
        print("Can't get token for", username)
        sys.exit()
    
    def get_saved_tracks():
        results = sp.current_user_saved_tracks()
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        return tracks

    tracks = get_saved_tracks()
    features = []
    for track in tracks:
        features.extend(sp.audio_features(track['track']['uri']))
    features = pd.DataFrame.from_records(features)
    features = features[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo', 'valence']]

else:
    features = pd.read_csv('sample_data') #TODO download the dataframe as a csv and add to path
    
features #TODO Add Name and Artist

What music would you like to include in the dataset?

Press 1 for saved/liked tracks
Press 2 for a sample dataset

Value: 1
Please Enter Username
Username: 12124976904


,acousticness,danceability,energy,instrumentalness,liveness,tempo,valence
0,0.880000,0.324,0.428,0.000013,0.1070,173.464,0.243
1,0.017500,0.810,0.731,0.000141,0.0655,143.969,0.622
2,0.011600,0.562,0.850,0.000000,0.1560,125.808,0.410
3,0.162000,0.725,0.680,0.000002,0.1580,119.939,0.760
4,0.001460,0.593,0.749,0.000069,0.3240,155.958,0.285
...,...,...,...,...,...,...,...
422,0.000056,0.579,0.865,0.000221,0.6490,104.990,0.358
423,0.469000,0.576,0.451,0.038900,0.0876,90.439,0.803
424,0.360000,0.671,0.740,0.000000,0.1180,86.932,0.908
425,0.150000,0.424,0.875,0.000000,0.2700,168.058,0.440


## Exploratory Data Analysis & Visualizations

In [48]:
#Explain the columns and how values are determined.
# Summary Statistics
# Histograms
# Correlation Graphs

## Machine Learning

In [ ]:
# Found cool article on PCA & One Class SVM here:
# https://medium.com/cuepoint/visualizing-hundreds-of-my-favorite-songs-on-spotify-fe50c94b8af3
# Implementation is found here:
# http://www.esparsons.com/blogmain/2018/11/27/an-eclectic-spotify-library-visualized-with-chartify

#TODO PCA Analysis
#TODO One Class SVM
#TODO Clustering

#TODO Error analysis...
#TODO Check if new playlists can fit into the SVM


## Takeaways 